Test NumPy multinomial vs. TensorFlow Probability Multinomial for <https://github.com/tensorflow/probability/issues/898>.

In [1]:
import tensorflow as tf # python -m pip install tf-nightly t
import tensorflow_probability as tfp # python -m pip install tfp-nightly
import numpy as np
print('tf', tf.__version__)
print('tfp', tfp.__version__)
print('np', np.__version__)

tf 2.2.0-dev20200427
tfp 0.11.0-dev20200427
np 1.18.1


In [11]:
N = 1000000
m = 50
n = 500

p = np.random.exponential(size=(n, m))
p /= p.sum(axis=1, keepdims=True)
assert np.allclose(1, p.sum(axis=1))
p[0]

array([0.00297214, 0.09104667, 0.05511024, 0.00113446, 0.04235687,
       0.01866667, 0.00870696, 0.01496359, 0.00307013, 0.01482515,
       0.01184055, 0.00320658, 0.00920683, 0.00413973, 0.03984834,
       0.00509918, 0.00130112, 0.02442503, 0.00072632, 0.01279097,
       0.00099365, 0.00637722, 0.00154611, 0.10655065, 0.03667299,
       0.00552926, 0.02204457, 0.02467754, 0.01182961, 0.02111125,
       0.0195045 , 0.03008724, 0.01481849, 0.06423313, 0.01717561,
       0.0131581 , 0.01485412, 0.02061338, 0.00381797, 0.0170003 ,
       0.00191953, 0.02103308, 0.00783909, 0.00856163, 0.0326934 ,
       0.02434709, 0.00560807, 0.00956336, 0.00378877, 0.06661277])

nump.random.multinomial

In [12]:
x = np.array([np.random.multinomial(N, p_) for p_ in p])
assert x.shape == p.shape
assert np.allclose(N, x.sum(axis=1))
print(x[0])

[  3053  90853  55026   1125  42034  18382   8886  14748   3076  14729
  11558   3283   9275   4078  39946   5210   1344  24443    748  12850
   1063   6379   1591 106357  36553   5560  22066  24648  12026  21110
  19751  30092  14777  64432  17232  13031  14735  20825   3948  17043
   1934  21137   7997   8591  32708  24213   5611   9627   3778  66538]


In [13]:
%timeit np.array([np.random.multinomial(n, p_) for p_ in p])

16.4 ms ± 751 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


[tfp.distributions.Multinomial](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/Multinomial?version=nightly)

In [14]:
dist = tfp.distributions.Multinomial(total_count=N, probs=p)

In [15]:
@tf.function
def sample():
    return dist.sample()

sample();

In [16]:
x = sample().numpy()
assert x.shape == p.shape
assert np.allclose(N, x.sum(axis=1))
print(x[0])

[  2955.  91228.  55171.   1128.  42534.  18664.   8752.  14856.   3069.
  15084.  11847.   3207.   9067.   4211.  39796.   4931.   1342.  24312.
    738.  12802.    966.   6414.   1597. 106271.  37053.   5523.  21975.
  24754.  11845.  20986.  19434.  30400.  14772.  64045.  17118.  13001.
  14780.  20597.   3824.  16997.   1944.  20780.   7782.   8430.  32751.
  24324.   5587.   9507.   3757.  67092.]


In [17]:
%timeit sample().numpy()

10.3 ms ± 2.76 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
